In [10]:
from utils import *
import numpy as np
import matplotlib.pyplot as plt
import pyqpanda as pq
import pyvqnet as pv
import random

%matplotlib inline

In [87]:
class QM(pv.nn.Module):
    def __init__(self, name=""):
        super().__init__(name)
        self.encode1 = pv.qnn.vqc.qcircuit.RX(wires=0)
        self.encode2 = pv.qnn.vqc.qcircuit.RY(wires=0)
        self.vqc1 = pv.qnn.vqc.qcircuit.RX(True, True, wires=1)
        self.vqc2 = pv.qnn.vqc.qcircuit.RY(True, True, wires=0)
        self.measure = pv.qnn.vqc.MeasureAll(
            obs=[
                {
                    "wires": [i],
                    "observables": ["X"],
                    "coefficient": [1],
                }
                for i in range(2)
            ]
        )
        self.device = pv.qnn.vqc.QMachine(2)

    def forward(self, x, *args, **kwargs):
        self.device.reset_states(x.shape[0])
        n = x.shape[1]
        re = pv.tensor.zeros(x.shape)
        for i in range(0, n, 2):
            self.encode1(params=x[:, [i]], q_machine=self.device)
            self.encode1(params=x[:, [i + 1]], q_machine=self.device)
            self.vqc1(q_machine=self.device)
            self.vqc2(q_machine=self.device)
            re[:, i : i + 2] = self.measure(q_machine=self.device)
        return re

In [34]:
epoch = 1000
batch = 16

In [100]:
m = QM()
print(sum(p.numel() for p in m.parameters()))
# print(m.parameters())
x = np.random.rand(batch, 16).astype("float32")
y_pred = m(x)
print(y_pred[0])
y = np.random.randint(0, 16, (batch,), dtype="int64")
print(y)
print(y_pred.argmax(1, False).to_numpy())

2
[-0.2155638,-0.0000001,-0.8381646,-0.0000001, 0.1037406, 0.0000001,
 -0.0516179, 0.0000001,-0.7399704, 0.0000002, 0.1208702, 0.0000001,
 -0.2078417, 0.0000002,-0.7836383, 0.0000001]
[12  8  4  7 15  9 10 11 13  6  9  0 11  6  6  9]
[10  6 10  0  6 12 10  0  4 10 10  0  6  0  0 10]


In [101]:
los = pv.nn.loss.CrossEntropyLoss()
# opt = pv.optim.SGD(m.parameters())
opt = pv.optim.Adam(m.parameters())
for e in range(1, epoch + 1):
    opt.zero_grad()
    print(f"epoch {e} \t loss ", end="")
    y_pred = m(x)
    loss = los(y, y_pred)
    print(loss.item())
    y_p = y_pred.argmax(1, False).to_numpy()
    s = sum(y_p == y)
    print(y)
    print(y_p)
    print(f"correct: {s}")
    print()
    if s == y.shape[0]:
        print("done")
        break
    loss.backward()
    opt._step()

epoch 1 	 loss 2.662214756011963
[12  8  4  7 15  9 10 11 13  6  9  0 11  6  6  9]
[10  6 10  0  6 12 10  0  4 10 10  0  6  0  0 10]
correct: 2

epoch 2 	 loss 2.657931327819824
[12  8  4  7 15  9 10 11 13  6  9  0 11  6  6  9]
[10  6 10  0  6 12 10  0  4 10 10  0  6  0  0  0]
correct: 2

epoch 3 	 loss 2.653822660446167
[12  8  4  7 15  9 10 11 13  6  9  0 11  6  6  9]
[ 4  6 10  0  6 12 10  0  4 10  0  0  6  0  0  0]
correct: 2

epoch 4 	 loss 2.6498944759368896
[12  8  4  7 15  9 10 11 13  6  9  0 11  6  6  9]
[ 4  6 10  0  6 12 10  0  4 10  0  0  6  0  0  0]
correct: 2

epoch 5 	 loss 2.6461539268493652
[12  8  4  7 15  9 10 11 13  6  9  0 11  6  6  9]
[ 4  6  6  0  6 12 10  0  4 10  0  0  6  0  0  0]
correct: 2

epoch 6 	 loss 2.642606496810913
[12  8  4  7 15  9 10 11 13  6  9  0 11  6  6  9]
[ 4  6  6  0  6 12 10  0  4 10  0  0  6  0 12  0]
correct: 2

epoch 7 	 loss 2.6392581462860107
[12  8  4  7 15  9 10 11 13  6  9  0 11  6  6  9]
[ 4  6  6  0  6 12 10  0  0 10  0  0  6  0 1

KeyboardInterrupt: 

In [15]:
from pyvqnet.qnn.vqc import T,QMachine,Hadamard
device = QMachine(4)
layer = Hadamard(wires=0)
batchsize = 1
device.reset_states(1)
layer(q_machine = device)
layer(q_machine = device)
print(device.states)

[[[[[0.9999999+0.j,0.       +0.j],
    [0.       +0.j,0.       +0.j]],

   [[0.       +0.j,0.       +0.j],
    [0.       +0.j,0.       +0.j]]],


  [[[0.       +0.j,0.       +0.j],
    [0.       +0.j,0.       +0.j]],

   [[0.       +0.j,0.       +0.j],
    [0.       +0.j,0.       +0.j]]]]]


In [2]:
from pyvqnet import nn
from collections import OrderedDict

# 使用Sequential创建一个小模型
model = nn.Sequential(
          nn.Conv2D(1,20,(5, 5)),
          nn.ReLu(),
          nn.Conv2D(20,64,(5, 5)),
          nn.ReLu()
        )
print(type(model.state_dict()))
for i,v in model.state_dict().items():
    print(i,v.numel())

<class 'collections.OrderedDict'>
0.weights 500
0.bias 20
2.weights 32000
2.bias 64
